In [ ]:
# Cell 1: Check GPU availability
import tensorflow as tf
import torch

print("🖥️ GPU Information:")
print("=" * 40)

# TensorFlow GPU check
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
print(f"Built with CUDA: {tf.test.is_built_with_cuda()}")

# PyTorch GPU check (for reference)
print(f"PyTorch CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Test GPU computation
with tf.device('/GPU:0'):
    a = tf.random.normal([1000, 1000])
    b = tf.random.normal([1000, 1000])
    c = tf.matmul(a, b)
    print("✅ GPU computation test passed!")

🖥️ GPU Information:
TensorFlow version: 2.18.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Built with CUDA: True
PyTorch CUDA available: True
GPU Name: Tesla T4
GPU Memory: 15.8 GB
✅ GPU computation test passed!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cell 3: Download Food-101 dataset directly
import urllib.request
import tarfile
import os

def download_food101_to_drive():
    drive_path = '/content/drive/MyDrive/food-101'  # Lokasi di Google Drive
    tar_path = '/content/drive/MyDrive/food-101.tar.gz'

    print("📥 Downloading Food-101 dataset...")

    url = "http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz"

    if not os.path.exists(tar_path):
        print("Downloading... This will take a few minutes.")
        urllib.request.urlretrieve(url, tar_path)
        print("✅ Download complete!")
    else:
        print("Dataset archive already exists in Drive!")

    # Extract dataset
    if not os.path.exists(drive_path):
        print("📂 Extracting dataset to Google Drive...")
        with tarfile.open(tar_path, 'r:gz') as tar:
            tar.extractall(path='/content/drive/MyDrive/')
        print("✅ Extraction complete!")
    else:
        print("Dataset already extracted in Drive!")

    print(f"Dataset location: {drive_path}")
    return drive_path

# Download and extract to Google Drive
dataset_path = download_food101_to_drive()

📥 Downloading Food-101 dataset...
Dataset archive already exists in Drive!
Dataset already extracted in Drive!
Dataset location: /content/drive/MyDrive/food-101


In [ ]:
# STEP 3: Import libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# STEP 4: Path setup
base_dir = "food-101"
train_dir = os.path.join(base_dir, "images")
image_size = (224, 224)
batch_size = 32

# STEP 5: Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [ ]:
# STEP 6: Build model (EfficientNetB0 for speed + accuracy)
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(101, activation='softmax')  # 101 classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# STEP 7: Train
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# STEP 8: Save model
model.save("food101_model.h5")

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
 381/2525 ━━━━━━━━━━━━━━━━━━━━ 15:24 431ms/step - accuracy: 0.0070 - loss: 4.6369

In [ ]:
# Cell 7: Optimized dataset creation for Colab
def create_colab_dataset():
    """
    Create optimized dataset pipeline for Google Colab
    """

    print("📊 Creating optimized dataset pipeline...")

    # Read classes
    with open(config.CLASSES_FILE, 'r') as f:
        all_classes = [line.strip() for line in f.readlines()]

    print(f"Number of classes: {len(all_classes)}")

    # Collect all image paths and labels
    image_paths = []
    labels = []

    for class_idx, class_name in enumerate(all_classes):
        class_dir = os.path.join(config.DATA_DIR, class_name)
        if os.path.exists(class_dir):
            class_images = []
            for img_file in os.listdir(class_dir):
                if img_file.lower().endswith(('.jpg', '.jpeg')):
                    img_path = os.path.join(class_dir, img_file)
                    if os.path.getsize(img_path) > 0:  # Check file is not empty
                        class_images.append(img_path)

            print(f"Class {class_name}: {len(class_images)} images")
            image_paths.extend(class_images)
            labels.extend([class_idx] * len(class_images))

    print(f"Total images: {len(image_paths)}")

    # Convert to arrays
    image_paths = np.array(image_paths)
    labels = tf.keras.utils.to_categorical(labels, num_classes=len(all_classes))

    # Split dataset
    total_samples = len(image_paths)
    train_size = int(0.8 * total_samples)

    # Shuffle
    indices = np.random.permutation(total_samples)
    train_indices = indices[:train_size]
    val_indices = indices[train_size:]

    # Create datasets
    train_paths = image_paths[train_indices]
    train_labels = labels[train_indices]
    val_paths = image_paths[val_indices]
    val_labels = labels[val_indices]

    # Preprocessing function
    def preprocess_image(image_path, label):
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [config.IMG_SIZE, config.IMG_SIZE])
        image = tf.cast(image, tf.float32) / 255.0
        return image, label

    # Training augmentation
    def train_augmentation(image, label):
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_brightness(image, 0.2)
        image = tf.image.random_contrast(image, 0.8, 1.2)
        image = tf.image.random_saturation(image, 0.8, 1.2)
        image = tf.image.random_hue(image, 0.1)

        # Random rotation
        image = tf.image.rot90(image, k=tf.random.uniform([], 0, 4, dtype=tf.int32))

        # Ensure values stay in valid range
        image = tf.clip_by_value(image, 0.0, 1.0)
        return image, label

    # Create TensorFlow datasets
    AUTOTUNE = tf.data.AUTOTUNE

    # Training dataset
    train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
    train_ds = train_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE)
    train_ds = train_ds.map(train_augmentation, num_parallel_calls=AUTOTUNE)
    train_ds = train_ds.batch(config.BATCH_SIZE)
    train_ds = train_ds.prefetch(AUTOTUNE)

    # Validation dataset
    val_ds = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
    val_ds = val_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE)
    val_ds = val_ds.batch(config.BATCH_SIZE)
    val_ds = val_ds.prefetch(AUTOTUNE)

    print(f"✅ Dataset created successfully!")
    print(f"Training samples: {len(train_paths)}")
    print(f"Validation samples: {len(val_paths)}")

    return train_ds, val_ds, all_classes

# Create dataset
train_dataset, val_dataset, class_names = create_colab_dataset()

📊 Creating optimized dataset pipeline...
Number of classes: 101
Class apple_pie: 1000 images
Class baby_back_ribs: 1000 images
Class baklava: 1000 images
Class beef_carpaccio: 1000 images
Class beef_tartare: 1000 images
Class beet_salad: 1000 images
Class beignets: 1000 images
Class bibimbap: 1000 images
Class bread_pudding: 1000 images
Class breakfast_burrito: 1000 images
Class bruschetta: 1000 images
Class caesar_salad: 1000 images
Class cannoli: 1000 images
Class caprese_salad: 1000 images
Class carrot_cake: 1000 images
Class ceviche: 1000 images
Class cheesecake: 1000 images
Class cheese_plate: 1000 images
Class chicken_curry: 1000 images
Class chicken_quesadilla: 1000 images
Class chicken_wings: 1000 images
Class chocolate_cake: 1000 images
Class chocolate_mousse: 1000 images
Class churros: 1000 images
Class clam_chowder: 1000 images
Class club_sandwich: 1000 images
Class crab_cakes: 1000 images
Class creme_brulee: 1000 images
Class croque_madame: 1000 images
Class cup_cakes: 1000

In [ ]:
# Cell 8: Create optimized model for Colab
def create_optimized_model():
    """
    Create model optimized for Colab GPU training
    """

    print("🏗️ Building optimized model...")

    # Use EfficientNet for better accuracy (optional)
    # base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Or use MobileNetV2 for faster training
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False

    inputs = tf.keras.Input(shape=(224, 224, 3))

    # Data augmentation layers (for consistency)
    x = tf.keras.layers.RandomFlip("horizontal")(inputs)
    x = tf.keras.layers.RandomRotation(0.1)(x)
    x = tf.keras.layers.RandomZoom(0.1)(x)

    # Base model
    x = base_model(x, training=False)
    x = GlobalAveragePooling2D()(x)

    # Classification head
    x = Dense(1024)(x)
    x = BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = Dropout(0.5)(x)

    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = Dropout(0.3)(x)

    # Output layer (float32 for mixed precision)
    outputs = Dense(len(class_names), activation='softmax', dtype='float32')(x)

    model = tf.keras.Model(inputs, outputs)

    print("✅ Model created!")
    print(f"Total parameters: {model.count_params():,}")

    return model, base_model

model, base_model = create_optimized_model()

🏗️ Building optimized model...
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
✅ Model created!
Total parameters: 4,152,485


In [ ]:
# Cell: Simplified training pipeline
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

def simple_training_pipeline():
    """
    Simplified training pipeline tanpa kompleksitas berlebihan
    """

    print("🚀 Starting simplified training...")

    # Phase 1: Frozen base model
    print("\n📚 PHASE 1: Training with frozen base model")
    print("="*50)

    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']  # Hapus top_5_accuracy jika menyebabkan masalah
    )

    # Simple callbacks
    callbacks_phase1 = [
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            verbose=1
        ),
        EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=1
        )
    ]

    # Train Phase 1
    print("Starting Phase 1 training...")
    history1 = model.fit(
        train_dataset,
        epochs=10,
        validation_data=val_dataset,
        callbacks=callbacks_phase1,
        verbose=1
    )

    # Phase 2: Fine-tuning
    print("\n🔧 PHASE 2: Fine-tuning")
    print("="*50)

    # Unfreeze base model
    base_model.trainable = True

    # Freeze early layers
    for layer in base_model.layers[:-20]:  # Reduced number
        layer.trainable = False

    # Recompile with lower LR
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Simple callbacks for Phase 2
    callbacks_phase2 = [
        ReduceLROnPlateau(
            monitor='val_accuracy',
            factor=0.5,
            patience=3,
            mode='max',
            verbose=1
        ),
        EarlyStopping(
            monitor='val_accuracy',
            patience=5,
            mode='max',
            restore_best_weights=True,
            verbose=1
        ),
        ModelCheckpoint(
            'best_model.h5',
            save_best_only=True,
            monitor='val_accuracy',
            mode='max',
            verbose=1
        )
    ]

    # Train Phase 2
    print("Starting Phase 2 training...")
    history2 = model.fit(
        train_dataset,
        epochs=15,
        validation_data=val_dataset,
        callbacks=callbacks_phase2,
        verbose=1
    )

    # Final evaluation
    print("\n📊 Final Results:")
    final_loss, final_acc = model.evaluate(val_dataset, verbose=0)
    print(f"Final Validation Accuracy: {final_acc:.4f} ({final_acc*100:.2f}%)")

    # Save model
    model.save('food_classifier_final.h5')
    print("✅ Model saved as 'food_classifier_final.h5'")

    # Try TFLite conversion
    try:
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        tflite_model = converter.convert()

        with open('food_classifier.tflite', 'wb') as f:
            f.write(tflite_model)
        print("✅ TFLite model saved as 'food_classifier.tflite'")
    except Exception as e:
        print(f"⚠️ TFLite conversion error: {e}")

    # Save labels
    with open('labels.txt', 'w') as f:
        for label in class_names:
            f.write(label + '\n')
    print("✅ Labels saved as 'labels.txt'")

    return model, history1, history2

# Run simplified training
model, hist1, hist2 = simple_training_pipeline()

🚀 Starting simplified training...

📚 PHASE 1: Training with frozen base model
Starting Phase 1 training...
Epoch 1/10
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 328s 251ms/step - accuracy: 0.2115 - loss: 3.4440 - val_accuracy: 0.4546 - val_loss: 2.1432 - learning_rate: 0.0010
Epoch 2/10
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 312s 246ms/step - accuracy: 0.3384 - loss: 2.6813 - val_accuracy: 0.4861 - val_loss: 2.0032 - learning_rate: 0.0010
Epoch 3/10
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 309s 236ms/step - accuracy: 0.3657 - loss: 2.5521 - val_accuracy: 0.4903 - val_loss: 1.9763 - learning_rate: 0.0010
Epoch 4/10
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 294s 233ms/step - accuracy: 0.3794 - loss: 2.4955 - val_accuracy: 0.5056 - val_loss: 1.9265 - learning_rate: 0.0010
Epoch 5/10
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 291s 230ms/step - accuracy: 0.3922 - loss: 2.4378 - val_accuracy: 0.5130 - val_loss: 1.8933 - learning_rate: 0.0010
Epoch 6/10
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 319s 229ms/step - accuracy: 0.3981 - loss: 2.4024 - val_accur

1263/1263 ━━━━━━━━━━━━━━━━━━━━ 300s 229ms/step - accuracy: 0.4027 - loss: 2.3887 - val_accuracy: 0.5346 - val_loss: 1.8022 - learning_rate: 1.0000e-04
Epoch 2/15
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step - accuracy: 0.4753 - loss: 2.0617
Epoch 2: val_accuracy improved from 0.53455 to 0.57772, saving model to best_model.h5


1263/1263 ━━━━━━━━━━━━━━━━━━━━ 330s 236ms/step - accuracy: 0.4753 - loss: 2.0617 - val_accuracy: 0.5777 - val_loss: 1.6286 - learning_rate: 1.0000e-04
Epoch 3/15
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step - accuracy: 0.5037 - loss: 1.9287
Epoch 3: val_accuracy improved from 0.57772 to 0.58629, saving model to best_model.h5


1263/1263 ━━━━━━━━━━━━━━━━━━━━ 296s 234ms/step - accuracy: 0.5037 - loss: 1.9287 - val_accuracy: 0.5863 - val_loss: 1.5970 - learning_rate: 1.0000e-04
Epoch 4/15
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - accuracy: 0.5268 - loss: 1.8450
Epoch 4: val_accuracy improved from 0.58629 to 0.59371, saving model to best_model.h5


1263/1263 ━━━━━━━━━━━━━━━━━━━━ 313s 227ms/step - accuracy: 0.5268 - loss: 1.8449 - val_accuracy: 0.5937 - val_loss: 1.5541 - learning_rate: 1.0000e-04
Epoch 5/15
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - accuracy: 0.5405 - loss: 1.7717
Epoch 5: val_accuracy improved from 0.59371 to 0.60629, saving model to best_model.h5


...

1263/1263 ━━━━━━━━━━━━━━━━━━━━ 322s 231ms/step - accuracy: 0.5914 - loss: 1.5502 - val_accuracy: 0.6334 - val_loss: 1.3963 - learning_rate: 1.0000e-04
Epoch 11/15
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - accuracy: 0.5975 - loss: 1.5227
Epoch 11: val_accuracy improved from 0.63337 to 0.64926, saving model to best_model.h5


1263/1263 ━━━━━━━━━━━━━━━━━━━━ 301s 238ms/step - accuracy: 0.5975 - loss: 1.5227 - val_accuracy: 0.6493 - val_loss: 1.3431 - learning_rate: 1.0000e-04
Epoch 12/15
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - accuracy: 0.6045 - loss: 1.4891
Epoch 12: val_accuracy did not improve from 0.64926
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 322s 238ms/step - accuracy: 0.6045 - loss: 1.4891 - val_accuracy: 0.6416 - val_loss: 1.3601 - learning_rate: 1.0000e-04
Epoch 13/15
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - accuracy: 0.6135 - loss: 1.4584
Epoch 13: val_accuracy improved from 0.64926 to 0.65074, saving model to best_model.h5


1263/1263 ━━━━━━━━━━━━━━━━━━━━ 294s 233ms/step - accuracy: 0.6135 - loss: 1.4584 - val_accuracy: 0.6507 - val_loss: 1.3294 - learning_rate: 1.0000e-04
Epoch 14/15
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - accuracy: 0.6193 - loss: 1.4282
Epoch 14: val_accuracy did not improve from 0.65074
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 327s 237ms/step - accuracy: 0.6193 - loss: 1.4282 - val_accuracy: 0.6424 - val_loss: 1.3670 - learning_rate: 1.0000e-04
Epoch 15/15
1263/1263 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - accuracy: 0.6265 - loss: 1.4115
Epoch 15: val_accuracy improved from 0.65074 to 0.65089, saving model to best_model.h5


1263/1263 ━━━━━━━━━━━━━━━━━━━━ 291s 231ms/step - accuracy: 0.6265 - loss: 1.4115 - val_accuracy: 0.6509 - val_loss: 1.3310 - learning_rate: 1.0000e-04
Restoring model weights from the end of the best epoch: 15.

📊 Final Results:


Final Validation Accuracy: 0.6509 (65.09%)
✅ Model saved as 'food_classifier_final.h5'
Saved artifact at '/tmp/tmp6isu5qvu'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_155')
Output Type:
  TensorSpec(shape=(None, 101), dtype=tf.float32, name=None)
Captures:
  137187261267216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137187256713680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137187256714640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137187261267408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137187261267792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137187261267600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137187256713488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137187256714448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137187256715792: TensorSpec(shape=(), dtype=tf.resource, name

In [ ]:
import tensorflow as tf
import numpy as np

def fix_tflite_conversion(model, train_dataset):
    """
    Fixed TFLite conversion dengan beberapa strategi
    """

    print("🔧 Fixing TFLite conversion...")

    # Strategy 1: Convert to float32 first
    print("\n📊 Strategy 1: Converting to float32...")
    try:
        # Clone model dengan float32
        model_float32 = tf.keras.models.clone_model(model)
        model_float32.set_weights(model.get_weights())

        # Compile dengan float32
        model_float32.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        # Convert dengan float32 model
        converter = tf.lite.TFLiteConverter.from_keras_model(model_float32)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]

        tflite_model = converter.convert()

        # Save TFLite model
        with open('food_classifier_fixed.tflite', 'wb') as f:
            f.write(tflite_model)

        print("✅ Strategy 1 successful! Model saved as 'food_classifier_fixed.tflite'")
        return tflite_model

    except Exception as e:
        print(f"❌ Strategy 1 failed: {e}")

    # Strategy 2: Use representative dataset for quantization
    print("\n📊 Strategy 2: Using representative dataset...")
    try:
        def representative_dataset():
            for batch in train_dataset.take(100):  # Take 100 batches
                images, _ = batch
                yield [images.numpy().astype(np.float32)]

        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_dataset
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.uint8
        converter.inference_output_type = tf.uint8

        tflite_model = converter.convert()

        with open('food_classifier_quantized.tflite', 'wb') as f:
            f.write(tflite_model)

        print("✅ Strategy 2 successful! Model saved as 'food_classifier_quantized.tflite'")
        return tflite_model

    except Exception as e:
        print(f"❌ Strategy 2 failed: {e}")

    # Strategy 3: Enable SELECT_TF_OPS (Flex ops)
    print("\n📊 Strategy 3: Using TensorFlow Lite with Flex ops...")
    try:
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,
            tf.lite.OpsSet.SELECT_TF_OPS  # Enable TensorFlow ops
        ]

        tflite_model = converter.convert()

        with open('food_classifier_flex.tflite', 'wb') as f:
            f.write(tflite_model)

        print("✅ Strategy 3 successful! Model saved as 'food_classifier_flex.tflite'")
        print("⚠️  Note: This model requires TensorFlow Lite with Flex ops support")
        return tflite_model

    except Exception as e:
        print(f"❌ Strategy 3 failed: {e}")

    # Strategy 4: Create a simpler model without problematic layers
    print("\n📊 Strategy 4: Creating simplified model...")
    try:
        # Create inference-only model without training layers
        inference_model = create_inference_model(model)

        converter = tf.lite.TFLiteConverter.from_keras_model(inference_model)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]

        tflite_model = converter.convert()

        with open('food_classifier_simplified.tflite', 'wb') as f:
            f.write(tflite_model)

        print("✅ Strategy 4 successful! Model saved as 'food_classifier_simplified.tflite'")
        return tflite_model

    except Exception as e:
        print(f"❌ Strategy 4 failed: {e}")

    print("❌ All conversion strategies failed. Keeping Keras model only.")
    return None

def create_inference_model(trained_model):
    """
    Create a clean inference model without training-specific layers
    """

    print("🏗️ Creating clean inference model...")

    # Create a new model with same architecture but without training layers
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

    # Base model
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )

    # Simple inference model
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    outputs = Dense(101, activation='softmax')(x)

    inference_model = tf.keras.Model(inputs, outputs)

    # Copy weights from trained model (carefully)
    try:
        # Get weights from trained model
        trained_weights = trained_model.get_weights()

        # Set weights to inference model
        # Skip augmentation layers and batch norm layers
        inference_model.set_weights(trained_weights[-6:])  # Last 6 layers (dense layers)

        print("✅ Weights transferred successfully")

    except Exception as e:
        print(f"⚠️ Weight transfer failed: {e}")
        print("Using random weights for inference model")

    return inference_model

# Run the fixed conversion
tflite_model = fix_tflite_conversion(model, train_dataset)

🔧 Fixing TFLite conversion...

📊 Strategy 1: Converting to float32...
Saved artifact at '/tmp/tmpltdzuinb'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_155')
Output Type:
  TensorSpec(shape=(None, 101), dtype=tf.float32, name=None)
Captures:
  137186902885392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137186902889616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137186902890384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137186902889808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137186902888848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137186902888656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137186902892112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137186902892880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137186902892688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13718690

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


❌ Strategy 2 failed: Could not translate MLIR to FlatBuffer.<unknown>:0: error: loc(callsite(callsite(fused["Conv2D:", "functional_1/mobilenetv2_1.00_224_1/Conv1_1/convolution@__inference_function_647594"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_648701"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"])): 'tf.Conv2D' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"]): called from
<unknown>:0: note: loc(callsite(callsite(fused["Conv2D:", "functional_1/mobilenetv2_1.00_224_1/Conv1_1/convolution@__inference_function_647594"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper_648701"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"])): see current operation: %171 = "tf.Conv2D"(%170, %158) <{data_format = "NHWC", dilations = [1, 1, 1, 1], explicit_paddings = [], padding = "SAME", strides =